# Transformers para Martin Fierro

Los dos principales objetivos de este tutorial son mostrar cómo es una arquitectura transformer y cómo usar las herramientas provistas por [Hugging Face](https://huggingface.co/). En este tutorial usaremos una implementación de [GPT2](https://openai.com/blog/better-language-models/) en español para generar (una vez más) texto similar al Martín Fierro, que se puede descargar [aquí](https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/martin_fierro.txt).

In [15]:
import torch
import numpy as np
import pandas as pd
import transformers
import os
import sys
import wget
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

## Parte 1: Usando transformers de Hugging Face

La librería transformers permite usar modelos ya entrenados en un pipeline que procesa el input, lo tokeniza y hace el forward pass para generar las predicciones.

Dada una tarea, pipeline descarga un modelo y tokenizador apropiados para la tarea. En este caso, especificamos un modelo preentrenado de GPT2 small en idioma español. 

In [2]:
from transformers import pipeline

generator = pipeline('text-generation', model="datificate/gpt2-small-spanish")

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at datificate/gpt2-small-spanish.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at datificate/gpt2-small-spanish.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [19]:
@interact(x='Sample', max_length=50)
def g(x, max_length):
    generator(x, max_length=max_length)


interactive(children=(Text(value='Sample', description='x'), IntSlider(value=50, description='max_length', max…

In [24]:
generator('Ahí dice: "los miminos bien"', max_length=100)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[{'generated_text': 'Ahí dice: "los miminos bien" cuando se trata de la madre de una niña, y "su relación se basa en un beso, un intercambio de gestos, dos conversaciones, un diálogo y momentos, las relaciones entre lo que es un gran acontecimiento que en los últimos 15 años. Y para los niños les gusta a los miminos que vienen de todo el mundo, y quieren que se cumplan todos mis deseos. No puedo imaginar por qué uno de ellos se va a ser con los mim'}]

## Parte 2: Finetunning con transformers

Ahora que ya hemos visto cómo funciona usar un modelo preentrenado vamos a hacer un finetune con los datos del Martin Fierro para un modelo preentrenado. Lo primero que debemos hacer es importar el modelo y el tokenizador.

In [3]:
from transformers import GPT2Tokenizer, GPT2Model

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('datificate/gpt2-small-spanish')

tokenized_seq = tokenizer("Yo nunca habia escuchado hablar de Innsmouth hasta")
print('inputs_ids: {}'.format(tokenized_seq['input_ids']))
print('attention_mask: {}'.format(tokenized_seq['attention_mask']))
tokens = tokenizer.convert_ids_to_tokens(tokenized_seq['input_ids'])
print('tokens: {}'.format(tokens))

inputs_ids: [10621, 2686, 498, 363, 27422, 6242, 258, 599, 10007, 15308, 657]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens: ['Yo', 'Ġnunca', 'Ġhab', 'ia', 'Ġescuchado', 'Ġhablar', 'Ġde', 'ĠIn', 'ns', 'mouth', 'Ġhasta']


El tokenizador nos convertirá los datos de texto plano a los ids necesarios para entrenar.

In [5]:
model = GPT2Model.from_pretrained('datificate/gpt2-small-spanish', from_tf=True)

ResourceExhaustedError: OOM when allocating tensor with shape[50257,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:AssignVariableOp]